# Output evaluación Leasing X años
## variables
* estimacion_precio_electricidad:dict  (Estimación precios sheet)
* desglose_consumo:dict (Consumo sheet)
* evaluacion_ppa_años:dict (Rentabilidad ESCO sheet)


* O&M_active: bool
* calculo_flujo_cliente:dict   (Cliente sheet)

* * Tasa leasing mensual
* * Plazo
* * Cuota mensual



* Falta que se realice inversión de planta por X años


In [21]:
import pandas as pd
import json
import matplotlib.pyplot as plt 
import numpy as np 

In [22]:
evaluacion_leasing= {}

In [23]:
with open('../variables/desglose_consumo.json') as archivo_consumo:
    datos_consumo_json=json.load(archivo_consumo)
datos_consumo=pd.DataFrame(datos_consumo_json)


with open('../variables/estimacion_precio_electricidad.json') as archivo_estimacion:
    datos_precio_electricidad_json=json.load(archivo_estimacion)
datos_precio_electricidad=pd.DataFrame(datos_precio_electricidad_json)


with open('../variables/evaluacion_ppa_anos.json') as archivo_evaluacion:
    datos_evaluacion_ppa_anos_json=json.load(archivo_evaluacion)
datos_evaluacion_ppa_anos=pd.DataFrame(datos_evaluacion_ppa_anos_json)


with open('../variables/calculo_flujo_cliente.json') as archivo_flujo_cliente:
    datos_flujo_cliente_json=json.load(archivo_flujo_cliente)
datos_flujo_cliente=pd.DataFrame(datos_flujo_cliente_json)


In [24]:
contador=0
tasa_leasing_mensual = 0.90/100
plazo=60
cuota_mensual=4712589
anos_leasing=1



for fecha in datos_consumo:
    #Por fecha
    datos_precio_electricidad_fecha=datos_precio_electricidad[fecha]
    datos_evaluacion_ppa_anos_fecha=datos_evaluacion_ppa_anos[fecha]
    datos_consumo_fecha=datos_consumo[fecha]

    #Variables
    inyeccion=datos_consumo_fecha["Inyeccion"]
    precio_estimado_energia=datos_precio_electricidad_fecha["Precio estm. nominal CLP energia"]
    costo_o_m_admin=datos_evaluacion_ppa_anos_fecha["Ingresos y Egresos"]["O&M y Admin"]
    margen_o_m= datos_flujo_cliente["Calculo flujo cliente"] ["Margen de O&M"]
    margen_o_m_porcentaje=margen_o_m/100
    consumo_distribuidora= datos_consumo_fecha["Consumo distribuidora"]
    precio_estimado_transporte =datos_precio_electricidad_fecha["Precio estm. nominal CLP Transporte"]
    consumo_cliente=datos_consumo_fecha["Consumo cliente"]




    #Calculos
    
    ingreso_inyeccion_FV = round(inyeccion * precio_estimado_energia)

    inversion_planta_fv=(cuota_mensual *12)*-1
    o_m_admin_fv= (costo_o_m_admin/(1-margen_o_m_porcentaje))
    pago_distribuidora= round(-1*(consumo_distribuidora * (precio_estimado_transporte + precio_estimado_energia)))

    pago_anual_energia_fv_negativo=ingreso_inyeccion_FV  + o_m_admin_fv + pago_distribuidora + inversion_planta_fv
    pago_anual_energia_sin_fv= round(consumo_cliente * (precio_estimado_transporte + precio_estimado_energia))
    pago_anual_energia_con_fv=round(pago_anual_energia_fv_negativo *-1)


    ahorro_anual= pago_anual_energia_sin_fv - pago_anual_energia_con_fv
    ahorro_acumulado = ahorro_anual
    if contador==0:
        ingreso_inyeccion_FV=0
        inversion_planta_fv=0
        pago_anual_energia_fv_negativo=0
        pago_anual_energia_con_fv=0
        ahorro_anual=0
        ahorro_acumulado=0
    else:
        fecha_anterior= str(int(fecha) - 1)
        evaluacion_fecha_anterior=evaluacion_leasing[fecha_anterior]
        ahorro_acumulado_anterior= evaluacion_fecha_anterior["Ahorro acumulado"]
        ahorro_acumulado=ahorro_anual + ahorro_acumulado_anterior






    evaluacion_leasing[fecha]=({
    "Ingresos por inyeccion FV": ingreso_inyeccion_FV,
    "Inversion planta FV": inversion_planta_fv,
    "O&M y admin FV":o_m_admin_fv,
    "Pago distribuidora": pago_distribuidora,
    "Pago anual energia con FV-": pago_anual_energia_fv_negativo,
    "Pago anual energia sin FV":pago_anual_energia_sin_fv,
    "Pago anual energia con FV+" : pago_anual_energia_con_fv,
    "Ahorro anual": ahorro_anual,
    "Ahorro acumulado": ahorro_acumulado,

    })
    
    with open('../evaluaciones/leasing/evaluacion.json', 'w') as file:
        json.dump(evaluacion_leasing, file, indent=4)
    contador+=1





In [25]:

with open('../evaluaciones/leasing/evaluacion.json') as archivo_leasing:
    datos_leasing_json=json.load(archivo_leasing)
datos_leasing=pd.DataFrame(datos_leasing_json)


In [26]:
datos_leasing

,2022,2023,2024
Ingresos por inyeccion FV,0.0,33566064.0,35259455.0
Inversion planta FV,0.0,-56551068.0,-56551068.0
O&M y admin FV,0.0,-4454935.0,-4454935.0
Pago distribuidora,0.0,-29376187.0,-31013555.0
Pago anual energia con FV-,0.0,-56816126.0,-56760103.0
Pago anual energia sin FV,0.0,46661979.0,49132722.0
Pago anual energia con FV+,0.0,56816126.0,56760103.0
Ahorro anual,0.0,-10154147.0,-7627381.0
Ahorro acumulado,0.0,-10154147.0,-17781528.0
